In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [4]:
import graphlab as gl
from graphlab import SGraph, Vertex, Edge, degree_counting, connected_components, SFrame
import re
import slate
import cPickle as pickle

In [5]:
from fenaroli_parser.build_network import build_weighted_graph, extract_backbone

In [6]:
with open('./data/first_ing_comp_dict.pkl' ,'r') as f:
    ing_flav_dict = pickle.load(f)
with open('./data/recipes.pkl' ,'r') as f:
    recipes = pickle.load(f)

In [120]:
flav_network = SGraph()
vert_list = []
edge_list = []
ingrds_not_seen = ing_flav_dict.keys()

for ingr, comp_list in ing_flav_dict.iteritems():
    ingrds_not_seen.remove(ingr)
    vert_list.append(Vertex(ingr, attr={'deg':0}))
    for ingrd in ingrds_not_seen:
        weight = len(set(ing_flav_dict[ingrd]).intersection(set(comp_list)))
        if weight > 0:
            edge_list.append(Edge(ingr, ingrd, attr={'weight':weight}))
    vert_list.append(Vertex(ingr))

flav_network = flav_network.add_vertices(vert_list)
flav_network = flav_network.add_edges(edge_list)

In [141]:
flvn = SGraph()
new_node_list = flav_network.vertices.fillna('deg', 0)
flvn = flvn.add_vertices(new_node_list).add_edges(edge_list)

In [142]:
flvn.show()

Canvas is updated and available in a tab in the default browser.


In [117]:
flav_network.summary()

{'num_edges': 94715, 'num_vertices': 2659}

In [118]:
flav_network.show()

Canvas is updated and available in a tab in the default browser.


In [24]:
degs = degree_counting.create(flav_network)

In [27]:
deg_graph = degs['graph']
deg_graph.show()

Canvas is updated and available in a tab in the default browser.


In [137]:
def degree_count_fn (src, edge, dst):
    src['deg'] += 1
    dst['deg'] += 1
    return (src, edge, dst)

In [228]:
fn_with_deg = flvn.triple_apply(degree_count_fn, mutated_fields=['deg'])

In [229]:
fn_with_deg.show()

Canvas is updated and available in a tab in the default browser.


In [7]:
import cProfile

In [9]:
def get_range(dictionary, begin, end):
  return dict(itertools.islice(dictionary.iteritems(), begin, end+1)) 

In [10]:
len(ing_flav_dict)/10

265.9

In [14]:
ing_flav_dict_sub = get_range(ing_flav_dict,0,500)

In [15]:
fnet = build_weighted_graph(ing_flav_dict_sub)

In [403]:
full_bb = extract_backbone(fnet)

Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 233, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 285, in _fixed_getinnerframes
    lines = ulinecache.getlines(file)[start:end]
  File "/usr/local/lib/python2.7/site-packages/IPython/utils/ulinecache.py", line 37, in getlines
    return [l.decode(encoding, 'replace') for l in lines]
  File "/usr/local/Cellar/python/2.7.10_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/encodings/utf_8.py", line 15, in decode
    def decode(input, errors='strict'):
KeyboardInterrupt


ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [ ]:
%prun extract_backbone(fnet)

In [ ]:
reduced_bb.show()
full_bb.show()